# T2.4 Extraer con selenium datos de licitacións do Concello da Coruña

Queremos facer un código que garde nunha BBDD a Microsoft SQL Server a información dos expedientes de licitación do Concello da Coruña e nun directorio (no actual) as capturas de pantalla de cada expediente.

Para montar o Microsoft SQL Server sigue a guía de: https://jfsanchez.es/docs/docker-sqlserver-microsoft/

Debes empregar selenium e a páxina: https://contrataciondelestado.es

Debes navegar por ela coma se indica no documento.

Por cada elemento (expediente), meterse nel e facer captura de pantalla (automáticamente, coa API de selenium).

**Entrega:**

    O código nun jupyter notebook.
    A BBDD exportada de Microsoft SQL Server.
    Un zip coas capturas de pantalla que fixo selenium.


In [123]:
from webdriver_manager.firefox import GeckoDriverManager
GeckoDriverManager().install()

'C:\\Users\\david.fernandezrebor\\.wdm\\drivers\\geckodriver\\win64\\v0.34.0\\geckodriver.exe'

In [124]:

import argparse
from enum import Enum
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
import time
import random

In [125]:
driver = webdriver.Firefox()
driver.get('https://contrataciondelestado.es')


In [127]:
elem = driver.find_element(By.XPATH, '//div[@class="card-item"][1]')
elem.click()



In [128]:
elem=driver.find_element(By.XPATH, '//span[text()="Licitaciones"]')
elem.click()

In [129]:
elem=driver.find_element(By.XPATH, '//span[text()="Búsqueda avanzada"]')
elem.click()

In [130]:
elem=driver.find_element(By.XPATH, '//span[text()="Seleccionar"]')
elem.click()

In [131]:
elem = driver.find_element(By.XPATH, '//div[text()="ENTIDADES LOCALES"]/../../td[@class="multiline"]')
elem.click()



In [132]:
elem = elem.find_elements(By.XPATH, '//div[text()="Galicia"]/../../td[@class="multiline"]')
elem[1].click()

In [133]:
elem = elem.find_element(By.XPATH, '//div[text()="A Coruña"]/../../td[@class="multiline"]')
elem.click()

AttributeError: 'list' object has no attribute 'find_element'